# Using huggingface's transformers
(containg pretrained models fot auto text sumerization)

In [ ]:
import streamlit as st
from transformers import pipeline
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import re
import torch
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig
BART_PATH = 'facebook/bart-large-cnn'
from rouge import Rouge
from bs4 import BeautifulSoup
import requests

#url to summarize
url = "https://www.nytimes.com/2021/04/30/technology/robot-surgery-surgeon.html"


#function to get url
def get_url(URL):
    r = requests.get(URL)
    soup = BeautifulSoup(r.text, 'html.parser')
    results = soup.find_all(['body'])
    text = [result.text for result in results]
    article = ' '.join(text)
    return article


#creating variable of text to be summarized with url
my_book = get_url(url)



#function to chunk text
def nest_sentences(document):
    nested = []
    sent = []
    length = 0
    for sentence in nltk.sent_tokenize(document):
        length += len(sentence)
        if length < 500:
            sent.append(sentence)
        else:
            nested.append(sent)
            sent = []
            length = 0

    if sent:
        nested.append(sent)

    return nested
#Creating variable for chunked text
nested = nest_sentences(my_book)

#function to generate summary

def generate_summary(nested_sentences):
    
    #model for summariation & tokenization
    bart_model = BartForConditionalGeneration.from_pretrained(BART_PATH, output_past=True)
    bart_tokenizer = BartTokenizer.from_pretrained(BART_PATH, output_past=True)
     
    # using cuda for summarization
    device = 'cuda'
    summaries = []
    for nested in nested_sentences:
        input_tokenized = bart_tokenizer.encode(' '.join(nested), truncation=True, return_tensors='pt')
        input_tokenized = input_tokenized.to(device)
        summary_ids = bart_model.to('cuda').generate(input_tokenized,
                                        length_penalty=3.0,
                                        min_length=30,
                                        max_length=100)
        output = [bart_tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]
        summaries.append(output)
    summaries = [sentence for sublist in summaries for sentence in sublist]
    return summaries

# generating 2 level summary

def finalSummary(chunked_text):
    summary_one = generate_summary(chunked_text)
    nested_summ = nest_sentences(' '.join(summary_one))
    final_summary = generate_summary(nested_summ)
    print(final_summary)

#Calling summary function on chunked text
finalSummary(nested)




## Import libraries

In [46]:
from transformers import pipeline
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import re
import torch
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig
BART_PATH = 'facebook/bart-large-cnn'
from rouge import Rouge
from bs4 import BeautifulSoup
import requests


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\makye\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\makye\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [44]:
def get_url(URL):
    r = requests.get(URL)
    soup = BeautifulSoup(r.text, 'html.parser')
    results = soup.find_all(['h1', 'p'])
    text = [result.text for result in results]
    article = ' '.join(text)
    return article

In [48]:
my_book = get_url("https://www.nytimes.com/2021/04/30/technology/robot-surgery-surgeon.html")
my_book

'Advertisement Supported by The Robot Surgeon Will See You Now Real scalpels, artificial intelligence — what could go wrong? By Cade Metz Sitting on a stool several feet from a long-armed robot, Dr. Danyal Fer wrapped his fingers around two metal handles near his chest. As he moved the handles — up and down, left and right — the robot mimicked each small motion with its own two arms. Then, when he pinched his thumb and forefinger together, one of the robot’s tiny claws did much the same. This is how surgeons like Dr. Fer have long used robots when operating on patients. They can remove a prostate from a patient while sitting at a computer console across the room. But after this brief demonstration, Dr. Fer and his fellow researchers at the University of California, Berkeley, showed how they hope to advance the state of the art. Dr. Fer let go of the handles, and a new kind of computer software took over. As he and the other researchers looked on, the robot started to move entirely on i

In [50]:
# def cleanText(text):
#     text = re.sub(r'\n|\r', ' ', text)
#     text = re.sub(r' +', ' ', text)
#     text = text.strip()
#     return text

# doc = cleanText(my_book)

In [68]:
def nest_sentences(document):
    nested = []
    sent = []
    length = 0
    for sentence in nltk.sent_tokenize(document):
        length += len(sentence)
        if length < 500:
            sent.append(sentence)
        else:
            nested.append(sent)
            sent = []
            length = 0

    if sent:
        nested.append(sent)

    return nested

In [70]:

nested = nest_sentences(my_book)
len(nested)
# nested[13]

15

In [72]:
%%time
bart_model = BartForConditionalGeneration.from_pretrained(BART_PATH, output_past=True)
bart_tokenizer = BartTokenizer.from_pretrained(BART_PATH, output_past=True)

Wall time: 14.1 s


In [73]:
def generate_summary(nested_sentences):
  device = 'cuda'
  summaries = []
  for nested in nested_sentences:
    input_tokenized = bart_tokenizer.encode(' '.join(nested), truncation=True, return_tensors='pt')
    input_tokenized = input_tokenized.to(device)
    summary_ids = bart_model.to('cuda').generate(input_tokenized,
                                      length_penalty=3.0,
                                      min_length=30,
                                      max_length=100)
    output = [bart_tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]
    summaries.append(output)
  summaries = [sentence for sublist in summaries for sentence in sublist]
  return summaries

In [74]:
%%time
summ = generate_summary(nested)

Wall time: 14.4 s


In [75]:
summ

['Dr. Danyal Fer wrapped his fingers around two metal handles near his chest. As he moved the handles, the robot mimicked each small motion with its own two arms.',
 'Researchers at the University of California, Berkeley, showed how they hope to advance the state of the art. Dr. Fer let go of the handles, and a new kind of computer software took over.',
 'The robot can match or even exceed a human in dexterity, precision and speed. The training exercise was originally designed for humans; moving the rings from peg to peg is how surgeons learn to operate robots.',
 'Using many of the same technologies that underpin self-driving cars, autonomous drones and warehouse robots, researchers are working to automate surgical robots too. “It is an exciting time,” said Russell Taylor, a professor at Johns Hopkins University.',
 'Robots can already exceed human accuracy on some surgical tasks. The hope is that automated robots can bring greater accuracy to other tasks, like incisions or suturing.'

## using pretrained models/pipeline for auto-text summerization

In [76]:
nested_summ = nest_sentences(' '.join(summ))
nested_summ

[['Dr. Danyal Fer wrapped his fingers around two metal handles near his chest.',
  'As he moved the handles, the robot mimicked each small motion with its own two arms.',
  'Researchers at the University of California, Berkeley, showed how they hope to advance the state of the art.',
  'Dr. Fer let go of the handles, and a new kind of computer software took over.',
  'The robot can match or even exceed a human in dexterity, precision and speed.'],
 ['Using many of the same technologies that underpin self-driving cars, autonomous drones and warehouse robots, researchers are working to automate surgical robots too.',
  '“It is an exciting time,” said Russell Taylor, a professor at Johns Hopkins University.',
  'Robots can already exceed human accuracy on some surgical tasks.',
  'The hope is that automated robots can bring greater accuracy to other tasks, like incisions or suturing.'],
 ['“We can’t automate the whole process, at least not without human oversight,” he said.',
  'Researche

In [77]:
%%time
model_output = generate_summary(nested_summ)

Wall time: 4.18 s


In [78]:
model_output

['Researchers at the University of California, Berkeley, showed how they hope to advance the state of the art. The robot can match or even exceed a human in dexterity, precision and speed.',
 'Researchers are working to automate surgical robots too. Robots can already exceed human accuracy on some surgical tasks. The hope is that automated robots can bring greater accuracy to other tasks.',
 'Researchers implanted tiny markers in the pig’s intestines that emitted a near-infrared light. The method is far from practical, as the markers are not easily implanted or removed.',
 'The images provide a detailed road map showing how surgeries are performed. They can help new surgeons understand how to use these robots. By analyzing images that show how a surgeon guides the robot, a neural network can learn the same skills.',
 'The robot could grab and move the plastics rings, matching the performance of human operators. But the automated system could not deal with change and uncertainty. Other 

In [30]:
# reference source: https://www.britannica.com/topic/Around-the-World-in-Eighty-Days-by-Verne
# Import refrence summary
ref_file = openFile('ref.txt')

#Clean refrence summary
ref_summary = cleanText(ref_file)

In [60]:
def Convert(string):
    li = list(string.split("."))
    return li

In [67]:
reference = Convert(ref_summary)
reference

['Phileas Fogg, a London gentleman of meticulous and unchanging habits, hires as his valet Jean Passepartout, a Frenchman who has had a variety of jobs, including circus performer, but now seeks a tranquil life',
 ' After reading in The Daily Telegraph that a new railroad in India has made it theoretically possible to travel around the world in 80 days, Fogg bets his fellow members at the Reform Club that he will make that journey in 80 days or less; the wager is for the princely sum of Â£20,000 (half his fortune)',
 ' Leaving that night, Fogg and a nonplussed Passepartout board a train bound for Dover and Calais to begin their journey',
 ' Shortly before Foggâ€™s departure, someone resembling him had robbed a bank, and Foggâ€™s sudden exit leads Scotland Yard to believe that he was the bank robber',
 ' Accordingly, a detective, Mr',
 ' Fix, is sent to Suez, in British-ruled Egypt, to await the steamer Mongolia, on which Fogg and Passepartout are traveling',
 ' Fix befriends Passpartou

In [62]:
rouge = Rouge()

In [72]:
rouge.get_scores(model_output, reference, avg=True, ignore_empty=True)

{'rouge-1': {'f': 0.10560533591410509,
  'p': 0.08825891971057485,
  'r': 0.15524981493748566},
 'rouge-2': {'f': 0.004530396097373422,
  'p': 0.004131064190382035,
  'r': 0.005952380952380952},
 'rouge-l': {'f': 0.09103945436289292,
  'p': 0.076742861433437,
  'r': 0.1305427567521709}}